In [ ]:
import pandas as pd
import numpy as np
import torch
from torchvision.datasets import Caltech256, Caltech101
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers

from library.taxonomy import Taxonomy
from library.models import UniversalResNetModel
from library.datasets import (
    Caltech256DataModule,
    Caltech101DataModule,
    CombinedDataModule,
)

# Load dataset information
caltech256_labels = Caltech256(root="datasets/caltech256", download=False).categories
caltech101_labels = Caltech101(root="datasets/caltech101", download=False).categories

print(f"Caltech-256 classes: {len(caltech256_labels)}")
print(f"Caltech-101 classes: {len(caltech101_labels)}")

# Reduce the precision of matrix multiplication to speed up training
torch.set_float32_matmul_precision("medium")

In [ ]:
# Load both taxonomies created from the real-world datasets
hypothesis_taxonomy = Taxonomy.load("taxonomies/caltech256_caltech101_hypothesis.pkl")
mcfp_taxonomy = Taxonomy.load("taxonomies/caltech256_caltech101_mcfp.pkl")

In [ ]:
# Configuration for Multi-Domain Training

# Training configuration
TRAIN = False  # Set to True to train model from scratch

# Create individual dataset modules
caltech101_dm = Caltech101DataModule(batch_size=32)
caltech256_dm = Caltech256DataModule(batch_size=32)

# Create combined data module with domain IDs
# Domain 0: Caltech-101, Domain 1: Caltech-256
dataset_module = CombinedDataModule(
    dataset_modules=[caltech101_dm, caltech256_dm],
    domain_ids=[0, 1],
    batch_size=64,
    num_workers=11,
)

dataset_name = "Caltech-101 + Caltech-256 (Multi-Domain)"

# Configuration for both taxonomies
taxonomies_config = {
    "hypothesis": {
        "taxonomy": hypothesis_taxonomy,
        "model_name": "universal-resnet50-hypothesis-multi-domain-min-val-loss",
        "logger_name": "universal_hypothesis_multi_domain",
    },
    "mcfp": {
        "taxonomy": mcfp_taxonomy,
        "model_name": "universal-resnet50-mcfp-multi-domain-min-val-loss",
        "logger_name": "universal_mcfp_multi_domain",
    },
}

In [ ]:
# Training configuration (shared for both models)
training_config = {
    "max_epochs": 50,
    "optim": "adamw",
    "optim_kwargs": {
        "lr": 0.00005,  # Reduced from 0.0001
        "weight_decay": 0.001,
        "betas": (0.9, 0.999),
        "eps": 1e-8,
    },
    "lr_scheduler": "cosine",  # Changed from multistep
    "lr_scheduler_kwargs": {
        "T_max": 50,  # matches max_epochs
        "eta_min": 1e-7,
    },
}

In [ ]:
# Train models for both taxonomies
results = {}

for taxonomy_name, config in taxonomies_config.items():
    # Create the Universal ResNet model for this taxonomy
    model = UniversalResNetModel(
        taxonomy=config["taxonomy"],
        architecture="resnet50",
        optim=training_config["optim"],
        optim_kwargs=training_config["optim_kwargs"],
        lr_scheduler=training_config["lr_scheduler"],
        lr_scheduler_kwargs=training_config["lr_scheduler_kwargs"],
    )

    # Setup trainer
    if TRAIN:
        tb_logger = pl_loggers.TensorBoardLogger(
            save_dir="logs", name=config["logger_name"]
        )

        trainer = Trainer(
            max_epochs=training_config["max_epochs"],
            logger=tb_logger,
            callbacks=[
                ModelCheckpoint(
                    dirpath="checkpoints",
                    monitor="val_accuracy",
                    mode="max",
                    save_top_k=1,
                    filename=config["model_name"],
                    enable_version_counter=False,
                )
            ],
        )

        # Train the model
        trainer.fit(model, datamodule=dataset_module)

        # Test the trained model
        test_results = trainer.test(datamodule=dataset_module, ckpt_path="best")

    else:
        trainer = Trainer(
            logger=False,
            enable_checkpointing=False,
        )

        # Load pre-trained model
        print(f"Loading pre-trained model: {config['model_name']}.ckpt")
        model = UniversalResNetModel.load_from_checkpoint(
            f"checkpoints/{config['model_name']}.ckpt",
            taxonomy=config[
                "taxonomy"
            ],  # Need to pass taxonomy since it's not serialized
        )

        # Test the loaded model
        test_results = trainer.test(model, datamodule=dataset_module)

    # Store results
    results[taxonomy_name] = test_results

In [ ]:
# Create individual combined data modules for each domain
# These maintain the (target, domain_id) tuple format expected by the universal models
caltech101_combined_dm = CombinedDataModule(
    dataset_modules=[caltech101_dm],
    domain_ids=[0],  # Domain 0 for Caltech-101
    batch_size=64,
    num_workers=11,
)

caltech256_combined_dm = CombinedDataModule(
    dataset_modules=[caltech256_dm],
    domain_ids=[1],  # Domain 1 for Caltech-256
    batch_size=64,
    num_workers=11,
)

# Test each model on individual domains
domain_results = {}
for taxonomy_name, config in taxonomies_config.items():
    # Load the trained model
    print(f"Loading pre-trained model: {config['model_name']}.ckpt")
    model = UniversalResNetModel.load_from_checkpoint(
        f"checkpoints/{config['model_name']}.ckpt", taxonomy=config["taxonomy"]
    )

    # Create trainer for testing
    trainer = Trainer(
        logger=False,
        enable_checkpointing=False,
    )

    domain_results[taxonomy_name] = {
        "name": taxonomy_name,
    }

    # Test on Caltech-101 (Domain 0)
    caltech101_results = trainer.test(model, datamodule=caltech101_combined_dm)
    domain_results[taxonomy_name]["caltech101"] = caltech101_results[0]["eval_accuracy"]

    # Test on Caltech-256 (Domain 1)
    caltech256_results = trainer.test(model, datamodule=caltech256_combined_dm)
    domain_results[taxonomy_name]["caltech256"] = caltech256_results[0]["eval_accuracy"]

    # Test on original test (both)
    original_results = trainer.test(model, datamodule=dataset_module)
    domain_results[taxonomy_name]["unified"] = original_results[0]["eval_accuracy"]

In [ ]:
# Create dataframe
df = pd.DataFrame.from_dict(domain_results, orient="index")

# Clear index
df.reset_index(drop=True, inplace=True)

# Print dataframe
print(df)

In [ ]:
# Calculate training duration for each taxonomy
from csv import DictReader
import datetime


def calculate_training_duration(file_prefix):
    """Calculate training duration from walltime in training CSV file"""
    with open(f"training_results/{file_prefix}_train.csv", "r") as f:
        reader = DictReader(f)
        rows = list(reader)

        if not rows:
            return "N/A"

        # Get first and last walltime
        start_time = float(rows[0]["Wall time"])
        end_time = float(rows[-1]["Wall time"])

        # Calculate duration in seconds
        duration_seconds = end_time - start_time

        # Convert to hours and minutes
        hours = int(duration_seconds // 3600)
        minutes = int((duration_seconds % 3600) // 60)

        if hours > 0:
            return f"{hours}h {minutes}m"
        else:
            return f"{minutes}m"


# Calculate training durations for both taxonomies
training_durations = {}
for taxonomy_name, config in taxonomies_config.items():
    duration = calculate_training_duration(config["logger_name"])
    training_durations[taxonomy_name] = duration
    print(f"{taxonomy_name.capitalize()} taxonomy training duration: {duration}")

# Add training duration to domain_results
for taxonomy_name in domain_results:
    domain_results[taxonomy_name]["training_time"] = training_durations[taxonomy_name]

In [ ]:
# Create LaTeX table from results
# Transform the dataframe to have better column names for the table
df_table = df.copy()
df_table.columns = [
    "Taxonomy Method",
    "Caltech-101",
    "Caltech-256",
    "Combined",
    "Training Time",
]

# Reorder to move training time after taxonomy method
df_table = df_table[
    ["Taxonomy Method", "Training Time", "Caltech-101", "Caltech-256", "Combined"]
]

# Single domain baseline accuracies (as percentages)
caltech101_baseline = 97.23  # 0.9723 * 100
caltech256_baseline = 75.65  # 0.7565 * 100

# Convert accuracy values to percentages and add delta values for domain columns
df_table["Combined"] = (df_table["Combined"] * 100).round(2)

# Add delta values for domain columns
for idx, row in df_table.iterrows():
    # Caltech-101 column with delta
    acc_101 = row["Caltech-101"] * 100
    delta_101 = acc_101 - caltech101_baseline
    sign_101 = "+" if delta_101 >= 0 else ""
    df_table.loc[idx, "Caltech-101"] = f"{acc_101:.2f} ({sign_101}{delta_101:.2f})"

    # Caltech-256 column with delta
    acc_256 = row["Caltech-256"] * 100
    delta_256 = acc_256 - caltech256_baseline
    sign_256 = "+" if delta_256 >= 0 else ""
    df_table.loc[idx, "Caltech-256"] = f"{acc_256:.2f} ({sign_256}{delta_256:.2f})"

# Create LaTeX table
latex_table = (
    df_table.style.hide(axis="index")
    .format({"Combined": "{:.2f}"})  # Only format Combined column as numeric
    .to_latex(
        caption="Universal model evaluation results on multi-domain Caltech-101 + Caltech-256 test dataset. Models were trained on both domains simultaneously and evaluated on individual domains as well as the combined test set. Domain accuracy values show performance compared to single-domain ResNet-50 baselines (Caltech-101: 97.23\\%, Caltech-256: 75.65\\%). All accuracy values are shown as percentages.",
        label="tab:universal_model_results",
        column_format="lcccc",
        position="ht",
        position_float="centering",
        hrules=True,
    )
)

# Save to file
with open("../thesis/figures/universal_model_results.tex", "w") as f:
    f.write(latex_table)

print("\nLaTeX table preview:")
print(latex_table)

In [ ]:
from csv import DictReader
import matplotlib

matplotlib.use("pgf")
import matplotlib.pyplot as plt

# LaTeX settings
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "EB Garamond",
        "font.size": 11,
        "pgf.texsystem": "lualatex",
    }
)

# Create subplot with 2 plots, one for each taxonomy
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Configuration for both taxonomies
taxonomy_configs = [
    {
        "name": "hypothesis",
        "title": "Hypothesis Taxonomy",
        "file_prefix": "universal_hypothesis_multi_domain",
    },
    {
        "name": "mcfp",
        "title": "MCFP Taxonomy",
        "file_prefix": "universal_mcfp_multi_domain",
    },
]

# Plot training curves for each taxonomy
for idx, config in enumerate(taxonomy_configs):
    ax = axes[idx]

    # Load training data
    with open(f"training_results/{config['file_prefix']}_train.csv", "r") as f:
        reader = DictReader(f)
        steps_train = []
        train = []
        for row in reader:
            steps_train.append(int(row["Step"]))
            train.append(float(row["Value"]))

    # Load validation data
    with open(f"training_results/{config['file_prefix']}_val.csv", "r") as f:
        reader = DictReader(f)
        steps_val = []
        val = []
        for row in reader:
            steps_val.append(int(row["Step"]))
            val.append(float(row["Value"]))

    # Plot training and validation curves
    ax.plot(steps_train, train, label="Train", color="blue")
    ax.plot(steps_val, val, label="Validation", color="red")

    ax.set_xlabel("Steps")
    ax.set_ylabel("Accuracy")
    ax.set_title(config["title"])
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(
    "../thesis/figures/universal_model_training_curves.pgf", bbox_inches="tight"
)
plt.show()